<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages-&amp;-config" data-toc-modified-id="Import-packages-&amp;-config-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages &amp; config</a></span></li><li><span><a href="#Prepare-parameters" data-toc-modified-id="Prepare-parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare parameters</a></span></li><li><span><a href="#SEIR-model" data-toc-modified-id="SEIR-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>SEIR model</a></span></li><li><span><a href="#Entry-point" data-toc-modified-id="Entry-point-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Entry point</a></span></li><li><span><a href="#Debugging" data-toc-modified-id="Debugging-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Debugging</a></span></li><li><span><a href="#Run-scenario" data-toc-modified-id="Run-scenario-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Run scenario</a></span></li></ul></div>

## Import packages & config

In [1]:
# import sys
# sys.path.append('/home/joaoc/SimulaCorona/.env/lib/python3.6/site-packages/')

import pandas as pd
import numpy as np
import plotly.express as px
import yaml
from scipy.integrate import odeint
from tqdm import tqdm
# from paths import *

%reload_ext autoreload
%autoreload 2

from datetime import datetime
date_time = datetime.today().strftime('%Y-%m-%d-%H-%M')

## Prepare parameters

In [82]:
def prepare_states(population_params, disease_params, config):
    """
    Estimate non explicity population initial states

    Params
    --------

    population_param: dict
           Explicit population parameters:
                    - N: population
                    - I: infected
                    - R: recovered
                    - D: deaths

    config: dict
            General configuration files with rules to estimate implicit parameters

    Returns
    --------
    dict
           Explicit and implicit population parameters ready to be applied in the `model` function
    """
    
    e_perc = (config['exposed']['doubling_rate'] - 1) * disease_params['incubation_period'] # 0.26 * 6 = 1.56
    exposed = population_params['I'] * config['infected']['i1_percentage'] * e_perc
       
    initial_pop_params = {
        'S': population_params['N'] - population_params['R'] - population_params['D'] - population_params['I'] - exposed,
        'E': exposed,
        'I1': population_params['I'] * config['infected']['i1_percentage'], # 85.5%
        'I2': population_params['I'] * config['infected']['i2_percentage'], # 12.5%
        'I3': population_params['I'] * config['infected']['i3_percentage'], # 2.5%
        'R': population_params['R'],
        'D': population_params['D']
    }
    
    return initial_pop_params

In [83]:
def prepare_params(disease_params, config, reproduction_rate, N):
    """
    Estimate non explicity disease parameters

    Params
    --------

    disease_params: dict
        Diseases parameters:
                   - ...

    config: dict
            General configuration files with rules to estimate implicit parameters

    Returns
    --------
    dict
           Explicit and implicit disease parameters ready to be applied in the `model` function
    """
    
    #     TODO - The functions indicated are these => But these are on Sources > Dynamic model parameters
    #     a=1/IncubPeriod OK
    #     g1=(1/DurMildInf) * FracMild => p1=(1/DurMildInf)*FracMild
    #     p1=(1/DurMildInf) - g1 => g1=(1/DurMildInf)-p1
    #     p2=(1/DurHosp) * (FracCritical/(FracSevere+FracCritical)) ??
    #     g2=(1/DurHosp) - p2 ??
    #     u=(1/TimeICUDeath) * (CFR/FracCritical) 
    #     g3=(1/TimeICUDeath) - u

    frac_severe_to_critical = config['infected']['i3_percentage'] / (config['infected']['i2_percentage'] + config['infected']['i3_percentage'])
    frac_critical_to_death = disease_params['fatality_ratio'] / config['infected']['i3_percentage']
    
    print(frac_severe_to_critical)
    
    parameters = {'sigma': 1 / disease_params['incubation_period'],
                  'gamma1': config['infected']['i1_percentage'] / disease_params['mild_duration'],
                  'p1': (1 - config['infected']['i1_percentage']) / disease_params['mild_duration'],
                  'gamma2': (1 - frac_severe_to_critical) / disease_params['severe_duration'],
                  'p2': frac_severe_to_critical / disease_params['severe_duration'], 
                  'mu': frac_critical_to_death / disease_params['critical_duration'], 
                  'gamma3': (1 - frac_critical_to_death) / disease_params['critical_duration'], 
                 }
    
    # Assuming beta1 with 0.9 * R0
    parameters['beta1'] = 0.9 * (1 / disease_params['mild_duration']) * reproduction_rate # / N
    
    # And beta2 = beta3 with 0.1 * R0
    x = (1 / disease_params['mild_duration']) * (1 / disease_params['severe_duration']) * (1 / disease_params['critical_duration'])
    y = parameters['p1'] * (1/disease_params['critical_duration']) + parameters['p1'] * parameters['p2']
    
    parameters['beta3'] = 0.1 * (x / y) * reproduction_rate # / N
    parameters['beta2'] = parameters['beta3']  
    
    return parameters

## SEIR model

In [92]:
def SEIR(y, t, N, model_params):
    """
    The SEIR model differential equations.
    
    Params
    --------
    y: dict
         Population parameters:
              - S: susceptible
              - E: exposed
              - I_1: infected mild
              - I_2: infected severe
              - I_3: infected critical
              - R: recovered
              - D: deaths
    N: int
            Population size
            
    model_params: int
           Parameters of model dynamic (transmission, progression, recovery and death rates)

    Return
    -------
    pd.DataFrame
            Evolution of population parameters.
    """

    S, E, I1, I2, I3, R, D = y
    
    # Exposition of susceptible rate
    exposition_rate = (model_params['beta1'] * I1) + (model_params['beta2'] * I2) + (model_params['beta3'] * I3)
    
    # Susceptible
    dSdt = - exposition_rate * S / N
    
    # Exposed
    dEdt = exposition_rate * S / N - model_params['sigma'] * E
    
    # Infected (mild)
    dI1dt = model_params['sigma'] * E - (model_params['gamma1'] + model_params['p1']) * I1
    
    # Infected (severe)
    dI2dt = model_params['p1'] * I1 - (model_params['gamma2'] + model_params['p2']) * I2
    
    # Infected (critical)
    dI3dt = model_params['p2'] * I2 - (model_params['gamma3'] + model_params['mu']) * I3
    
    # Recovered
    dRdt = model_params['gamma1'] * I1 + model_params['gamma2'] * I2 + model_params['gamma3'] * I3
    
    # Deaths
    dDdt = model_params['mu'] * I3
     
    return dSdt, dEdt, dI1dt, dI2dt, dI3dt, dRdt, dDdt

In [97]:
# def model(initial_pop_params, model_params, N, n_days=90): 
#     """ Run X epidemiological model

#     Params
#     --------
#     initial_pop_params: dict
#          Population parameters:
#               - S: susceptible
#               - E: exposed
#               - I_1: infected mild
#               - I_2: infected severe
#               - I_3: infected critical
#               - R: recovered
#               - D: deaths

#     diseases_params: dict
#            Diseases parameters:
#                - ...
#     N: int
#             Population size
            
#     n_days: int
#            Number of days to model

#     Return
#     -------
#     pd.DataFrame
#             Evolution of population parameters.
#     """
    
#     t = np.linspace(0, n_days, n_days+1)
#     y0 = list(initial_pop_params.values())
    
#     params = {'y0': y0, 't': t, 'args': (N, model_params)}
    
#     result = odeint(SEIR, **params)
#     result = pd.DataFrame(result, columns=['S', 'E' ,'I1', 'I2', 'I3', 'R', 'D'])
#     # result['t'] = t
    
#     return result

## Entry point

In [93]:
def entrypoint(population_params, reproduction_rate=3, n_days=90):
    """
    Function to receive user input and run model.
    
    Params
    --------
    population_param: dict
           Explicit population parameters:
                    - N: population
                    - I: infected
                    - R: recovered
                    - D: deaths
                    
    path: str
        Current path from call.
                                 
    reproduction_rate: int
            Basic reproduction rate of the disease.
            
    n_days: int
            Number of days to model.

    Return
    -------
    pd.DataFrame
            Evolution of population parameters.
    """
    
    # Get config values
    model_parameters = yaml.load(open('../configs/model_parameters.yaml', 'r'), Loader=yaml.FullLoader)
    config = yaml.load(open('../configs/config.yaml', 'r'), Loader=yaml.FullLoader)
    
    # Prepate parameters
    initial_pop_params = prepare_states(population_params, 
                                        model_parameters['diseases_parameters'], config)
    seir_parameters = prepare_params(model_parameters['diseases_parameters'], config, 
                                     reproduction_rate, N=population_params['N'])
    
    print(seir_parameters)
    
    # Run model
    params = {'y0': list(initial_pop_params.values()),
              't': np.linspace(0, n_days, n_days+1), 
              'args': (population_params['N'], seir_parameters)}
    
    result = pd.DataFrame(odeint(SEIR, **params), 
                          columns=['S', 'E' ,'I1', 'I2', 'I3', 'R', 'D'])
    
    result['N'] = result.sum(axis=1)
    result['t'] = result.index
    
    return result

In [94]:
config = yaml.load(open('../configs/config.yaml', 'r'), Loader=yaml.FullLoader)
model_parameters = yaml.load(open('../configs/model_parameters.yaml', 'r'), Loader=yaml.FullLoader)

model_parameters, config

({'population_parameters': {'N': 10000, 'I': 1000, 'R': 0, 'D': 10},
  'diseases_parameters': {'mild_proportion': 0.8,
   'mild_duration': 6,
   'severe_proportion': 0.15,
   'severe_duration': 6,
   'critical_proportion': 0.05,
   'critical_duration': 8,
   'fatality_ratio': 0.02,
   'transmission_severe': 0.1,
   'transmission_critical': 0.1,
   'doubling_rate': 1.26,
   'incubation_period': 6},
  'estimation_days': 90},
 {'infected': {'i1_percentage': 0.855,
   'i2_percentage': 0.12,
   'i3_percentage': 0.025},
  'exposed': {'doubling_rate': 1.26}})

In [95]:
result = entrypoint(model_parameters['population_parameters'], reproduction_rate=3)
result

0.1724137931034483
{'sigma': 0.16666666666666666, 'gamma1': 0.1425, 'p1': 0.02416666666666667, 'gamma2': 0.13793103448275862, 'p2': 0.02873563218390805, 'mu': 0.09999999999999999, 'gamma3': 0.02500000000000001, 'beta1': 0.44999999999999996, 'beta3': 0.28037383177570085, 'beta2': 0.28037383177570085}


,S,E,I1,I2,I3,R,D,N,t
0,7656.200000,1333.800000,855.000000,120.000000,25.000000,0.000000,10.000000,10000.0,0
1,7323.809245,1435.373570,936.566297,121.537105,25.321222,144.876536,12.516026,10000.0,1
2,6979.192978,1532.590390,1020.875483,124.687482,25.668918,301.919610,15.065139,10000.0,2
3,6624.557036,1624.080012,1106.734061,129.250925,26.080786,471.645222,17.651959,10000.0,3
4,6262.629751,1708.202686,1192.897962,135.030885,26.584739,654.369602,20.284375,10000.0,4
...,...,...,...,...,...,...,...,...,...
86,443.932549,2.311992,5.695430,1.993035,1.526110,9311.623748,232.917135,10000.0,86
87,443.782427,2.095151,5.158629,1.807609,1.398027,9312.694904,233.063253,10000.0,87
88,443.646284,1.898740,4.672586,1.639291,1.280225,9313.665792,233.197083,10000.0,88
89,443.522816,1.720821,4.232489,1.486527,1.171936,9314.545796,233.319615,10000.0,89


In [98]:
px.line(result.melt('t'), x='t', y='value', color='variable')

## Debugging

In [72]:
initial_pop_params = prepare_states(model_parameters['population_parameters'], 
                                    model_parameters['diseases_parameters'], 
                                    config)
initial_pop_params

{'S': 7656.2,
 'E': 1333.8,
 'I1': 855.0,
 'I2': 120.0,
 'I3': 25.0,
 'R': 0,
 'D': 10}

In [73]:
initial_pop_params['I1'] + initial_pop_params['I2'] + initial_pop_params['I3']

1000.0

In [74]:
seir_parameters = prepare_params(model_parameters['diseases_parameters'], config, 
                                    reproduction_rate=5, N=model_parameters['population_parameters']['N'])
seir_parameters

0.1724137931034483


{'sigma': 0.16666666666666666,
 'gamma1': 0.1425,
 'p1': 0.02416666666666667,
 'gamma2': 0.13793103448275862,
 'p2': 0.02873563218390805,
 'mu': 0.09999999999999999,
 'gamma3': 0.02500000000000001,
 'beta1': 0.75,
 'beta3': 0.46728971962616817,
 'beta2': 0.46728971962616817}

In [75]:
result = model(initial_pop_params, seir_parameters, 
          N=model_parameters['population_parameters']['N'], n_days=model_parameters['estimation_days'])

# result['N'] = result.sum(axis=1)
result['t'] = result.index
result

,S,E,I1,I2,I3,R,D,t
0,7656.200000,1333.800000,855.000000,120.000000,25.000000,0.000000,10.000000,0
1,7107.310400,1634.804512,952.711937,121.665353,25.322143,145.669611,12.516045,1
2,6541.204217,1905.522510,1079.114782,125.596457,25.681953,307.814398,15.065683,2
3,5957.050939,2151.264561,1225.688444,131.982448,26.139345,490.218570,17.655694,3
4,5360.840797,2370.275767,1385.431244,140.813774,26.749410,695.590358,20.298651,4
...,...,...,...,...,...,...,...,...
86,54.494177,0.146039,0.691447,0.421477,0.633229,9701.833259,241.780373,86
87,54.488991,0.128387,0.606295,0.371170,0.569500,9701.995202,241.840455,87
88,54.484406,0.112893,0.531679,0.326813,0.511984,9702.137745,241.894480,88
89,54.480353,0.099290,0.466292,0.287712,0.460102,9702.263212,241.943040,89


In [76]:
result['I'] = result['I1'] + result['I2'] + result['I3']

In [77]:
px.line(result.melt('t'), x='t', y='value', color='variable')

## Run scenario